In [1]:
# Uncomment this if you are unable to download the dataset_url
#!pip install opendatasets

In [2]:
# /*To download data set*/

import opendatasets as od
dataset_url = 'https://www.kaggle.com/datasets/prashant268/chest-xray-covid19-pneumonia'
od.download(dataset_url)

Skipping, found downloaded files in "./chest-xray-covid19-pneumonia" (use force=True to force download)


In [5]:
dataroot = 'chest-xray-covid19-pneumonia/Data/train/'
#  NORMAL , PNEUMONIA
import os
#Copy the directory name for training
os.listdir(dataroot)

['COVID19', 'PNEUMONIA', 'NORMAL']

In [6]:
import torch
import torch.nn as nn
import shutil
import matplotlib.pyplot as plt

In [16]:
class Discriminator(nn.Module):
    def __init__(self, channels_img, features_d):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            nn.Conv2d(
                channels_img, features_d, kernel_size=4, stride=2, padding=1
            ),
            nn.LeakyReLU(0.2),
            self._block(features_d, features_d * 2, 4, 2, 1),
            self._block(features_d * 2, features_d * 4, 4, 2, 1),
            self._block(features_d * 4, features_d * 8, 4, 2, 1),
            nn.Conv2d(features_d * 8, 1, kernel_size=4, stride=2, padding=0),
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False,
            ),
            nn.InstanceNorm2d(out_channels, affine=True),
            nn.LeakyReLU(0.2),
        )

    def forward(self, x):
        return self.disc(x)


class Generator(nn.Module):
    def __init__(self, channels_noise, channels_img, features_g):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            self._block(channels_noise, features_g * 32, 4, 1, 0),
            self._block(features_g * 32, features_g * 16, 4, 1, 0),
            self._block(features_g * 16, features_g * 8, 4, 2, 1),
            self._block(features_g * 8, features_g * 4, 4, 2, 1),
            self._block(features_g * 4, features_g * 2, 4, 2, 1),
            nn.ConvTranspose2d(
                features_g * 2, channels_img, kernel_size=4, stride=2, padding=1
            ),
            nn.Tanh(),
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False,
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.net(x)


def initialize_weights(model):
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data, 0.0, 0.02)

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"
LEARNING_RATE = 5e-5
BATCH_SIZE = 64
IMAGE_SIZE = 64
CHANNELS_IMG = 3
Z_DIM = 128
NUM_EPOCHS = 10
FEATURES_CRITIC = 64
FEATURES_GEN = 64
CRITIC_ITERATIONS = 5
WEIGHT_CLIP = 0.01

In [19]:
transform = transforms.Compose(
    [
        transforms.Resize((IMAGE_SIZE,IMAGE_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.5 for _ in range(CHANNELS_IMG)], [0.5 for _ in range(CHANNELS_IMG)]
        ),
    ]
)


# # Define the transformations you want to apply to the images
#  transform = transforms.Compose([
#    transforms.Resize((64, 64)),  # Resize images to a consistent size (for example 64x64)
#    transforms.ToTensor(),       # Convert images to tensors
 #   transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
#])



In [20]:
#dataset = datasets.ImageFolder(root=dataroot, transform=transforms)
#loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [21]:
# Load the entire dataset
full_dataset = datasets.ImageFolder(root=dataroot, transform=transform)

# Filter the dataset to include only images from the "COVID19" class folder
class_folder = 'COVID19'
class_index = full_dataset.class_to_idx[class_folder]
filtered_indices = [i for i in range(len(full_dataset)) if full_dataset[i][1] == class_index]

# Create a Subset dataset containing only the filtered images
from torch.utils.data import Subset
subset_dataset = Subset(full_dataset, filtered_indices)

# Create a DataLoader to iterate over the subset dataset in batches during training
BATCH_SIZE = 64
loader = DataLoader(subset_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [14]:
gen = Generator(Z_DIM, CHANNELS_IMG, FEATURES_GEN).to(device)
critic = Discriminator(CHANNELS_IMG, FEATURES_CRITIC).to(device)
initialize_weights(gen)
initialize_weights(critic)
# ============================above this part not req in case training  loaded model ================

opt_gen = optim.RMSprop(gen.parameters(), lr=LEARNING_RATE)
opt_critic = optim.RMSprop(critic.parameters(), lr=LEARNING_RATE)

fixed_noise = torch.randn(32, Z_DIM, 1, 1).to(device)
writer_real = SummaryWriter(f"logs2/real")
writer_fake = SummaryWriter(f"logs2/fake")
step = 0

In [27]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasRegressor


# Define the WGAN model function
def build_wgan_model(discriminator_optimizer='adam', generator_optimizer='adam', leaky_relu_alpha=0.2):
    # Generator
    Generator = keras.Sequential([
        keras.layers.Dense(256, input_dim=100),
        keras.layers.LeakyReLU(alpha=leaky_relu_alpha),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(512),
        keras.layers.LeakyReLU(alpha=leaky_relu_alpha),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(1024),
        keras.layers.LeakyReLU(alpha=leaky_relu_alpha),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(784, activation='tanh')
    ])

    # Discriminator
    Discriminator = keras.Sequential([
        keras.layers.Dense(1024, input_dim=784),
        keras.layers.LeakyReLU(alpha=leaky_relu_alpha),
        keras.layers.Dense(512),
        keras.layers.LeakyReLU(alpha=leaky_relu_alpha),
        keras.layers.Dense(256),
        keras.layers.LeakyReLU(alpha=leaky_relu_alpha),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    # Compile discriminator
    Discriminator.compile(loss='binary_crossentropy', optimizer=discriminator_optimizer)

    # The WGAN model
    z = keras.layers.Input(shape=(100,))
    img = Generator(z)
    Discriminator.trainable = False
    valid = Discriminator(img)
    wgan = keras.Model(z, valid)
    wgan.compile(loss='binary_crossentropy', optimizer=generator_optimizer)

    return wgan

# Create the KerasRegressor for Grid Search
wgan_model = KerasRegressor(build_fn=build_wgan_model, epochs=10, batch_size=32, verbose=0)

# Define the hyperparameters to search through
param_grid = {
    'leaky_relu_alpha': [0.2, 0.3, 0.4],
    'discriminator_optimizer': ['adam', 'rmsprop'],
    'generator_optimizer': ['adam', 'rmsprop']
}

# Data Tracing
# image_data = np.array()
# labels = np.array(class_index)

# Split the data into training, validation and testing sets
# X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(estimator=wgan_model, param_grid=param_grid, n_jobs=-1)
grid_result = grid_search.fit(X_train, y_train)  # X_train and y_train are your data and labels

# Print the best hyperparameters found
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


/tmp/ipykernel_1610/3678043425.py:50: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  wgan_model = KerasRegressor(build_fn=build_wgan_model, epochs=10, batch_size=32, verbose=0)


TypeError: array() missing required argument 'object' (pos 0)

In [15]:
gen.train()
critic.train()

Discriminator(
  (disc): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (3): Sequential(
      (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (4): Sequential(
      (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (5): Conv2d(512, 1, kernel_size=(4, 4), stride=(2, 2))
  )
)

In [16]:
%load_ext tensorboard

In [17]:
# If you again load the trained model to retrain it 

#gen = Generator(Z_DIM, CHANNELS_IMG, FEATURES_GEN).to(device)
#gen.load_state_dict(torch.load('/content/drive/MyDrive/GAN128.pth', map_location='cpu'))

# critic = Discriminator(CHANNELS_IMG, FEATURES_CRITIC).to(device)
# critic.load_state_dict(torch.load('/content/drive/MyDrive/Diff-x1r.pth', map_location='cpu'))

In [18]:
import torchvision.utils as vutils
j = int(32)
img_list = []
for epoch in range(500):
    for batch_idx, (data, _) in enumerate(loader):
        data = data.to(device)
        cur_batch_size = data.shape[0]
        for _ in range(CRITIC_ITERATIONS):
            noise = torch.randn(cur_batch_size, Z_DIM, 1, 1).to(device)
            fake = gen(noise)
            critic_real = critic(data).reshape(-1)
            critic_fake = critic(fake).reshape(-1)
            loss_critic = -(torch.mean(critic_real) - torch.mean(critic_fake))
            critic.zero_grad()
            loss_critic.backward(retain_graph=True)
            opt_critic.step()
            for p in critic.parameters():
                p.data.clamp_(-WEIGHT_CLIP, WEIGHT_CLIP)
        gen_fake = critic(fake).reshape(-1)
        loss_gen = -torch.mean(gen_fake)
        gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()
        if batch_idx % 20 == 0 and batch_idx > 0:
            gen.eval()
            critic.eval()

            with torch.no_grad():
                fake = gen(noise)
                img_grid_real = torchvision.utils.make_grid(
                    data[:32], normalize=True
                )
                img_grid_fake = torchvision.utils.make_grid(
                    fake[:32], normalize=True
                )
                i = int(0)
                torchvision.utils.save_image(img_grid_fake, "generated_image/%04s.jpg" %step)

            step += 1
            gen.train()
            critic.train()

In [19]:
torch.save(gen.state_dict(), 'ank_covid_gan_128.pth')

In [13]:
#import numpy as np
#from sklearn.datasets import load_iris
#from sklearn.model_selection import train_test_split
#from sklearn.svm import SVC
#from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
#from sklearn.metrics import accuracy_score

In [22]:
 #Load the iris dataset
#data = load_iris()
#X = dataroot.data
#y = dataroot.target
# Split the data into training, validation, and test sets
#X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
#X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [ ]:
"""  param_grid = {
    'C': [0.1, 1, 10],        # Regularization parameter
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'gamma': [0.1, 1, 'scale', 'auto']
}
"""

In [ ]:
""" # Initialize the SVM classifier
svm = SVC()
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params_grid = grid_search.best_params_

# Train the model with the best hyperparameters on the combined training and validation sets
best_svm_grid = SVC(**best_params_grid)
best_svm_grid.fit(np.vstack((X_train, X_val)), np.hstack((y_train, y_val)))

# Evaluate on the test set
y_pred_grid = best_svm_grid.predict(X_test)
accuracy_grid = accuracy_score(y_test, y_pred_grid)
print("Accuracy (Grid Search):", accuracy_grid)
"""